In [1]:
if 0 :
    %matplotlib inline
else :
    %matplotlib notebook

#  Import  libraries

In [2]:
import sys
import os
module_path = os.path.abspath('.') +"\\_scripts"
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)
from _00_Import_packages_git3 import *

C:\Users\ng947ac\Documents\Daniel\jupyter\SoSTrades\jupyter_doc\_scripts
C:\Users\ng947ac\Documents\Daniel\git\workspace
C:\Users\ng947ac\Documents\Daniel\git\workspace\sosgemseo\src\
C:\Users\ng947ac\Documents\Daniel\git\workspace\sostrades-core


In [3]:
from numpy import array
import pandas as pd
from sos_trades_core.execution_engine.execution_engine import ExecutionEngine
from numpy.testing import assert_array_equal, assert_array_almost_equal  # @UnresolvedImport
import os
from gemseo.core.mdo_scenario import MDOScenario

# SoSDoeScenario setUp

In [4]:
study_name = 'doe'
ns = f'{study_name}'
sc_name = "SellarDoeScenario"
c_name = "SellarCoupling"
print(sc_name)

SellarDoeScenario


See §2 for Discipmline description

Here we are using another process with mda:

In [5]:
dspace_dict = {'variable': ['x', 'z'],
               'value': [[1.], [5., 2.]],
               'lower_bnd': [[0.], [-10., 0.]],
               'upper_bnd': [[10.], [10., 10.]],
               'enable_variable': [True, True],
               'activated_elem': [[True], [True, True]]}

In [6]:
dspace = pd.DataFrame(dspace_dict)
dspace

,variable,value,lower_bnd,upper_bnd,enable_variable,activated_elem
0,x,[1.0],[0.0],[10.0],True,[True]
1,z,"[5.0, 2.0]","[-10.0, 0.0]","[10.0, 10.0]",True,"[True, True]"


In [7]:
from tempfile import gettempdir
root_dir = gettempdir()
print(root_dir)
dspace.to_csv(root_dir+'\\ds.csv')

C:\Temp


In [8]:
repo = 'sos_trades_core.sos_processes.test'
proc_name =  'test_sellar_doe_discopt'

# Doe scenario execution

# 1 Set Doe optim scenario lhs or fullfact

In [9]:
my_formulation = "DisciplinaryOpt"
#my_formulation = "MDF"
#my_formulation = "IDF"
#my_formulation =  "BiLevel "

In [10]:
from gemseo.api import (
    configure_logger,
    get_algorithm_options_schema,
    get_available_doe_algorithms,
)
get_available_doe_algorithms()

['CustomDOE',
 'DiagonalDOE',
 'OT_SOBOL',
 'OT_RANDOM',
 'OT_HASELGROVE',
 'OT_REVERSE_HALTON',
 'OT_HALTON',
 'OT_FAURE',
 'OT_MONTE_CARLO',
 'OT_FACTORIAL',
 'OT_COMPOSITE',
 'OT_AXIAL',
 'OT_OPT_LHS',
 'OT_LHS',
 'OT_LHSC',
 'OT_FULLFACT',
 'OT_SOBOL_INDICES',
 'fullfact',
 'ff2n',
 'pbdesign',
 'bbdesign',
 'ccdesign',
 'lhs']

In [11]:
my_doe_algo = "lhs"
#my_doe_algo = "fullfact"

In [12]:
print("\n Sellar doe solution check with ",my_formulation ," formulation")


 Sellar doe solution check with  DisciplinaryOpt  formulation


In [13]:
exec_eng = ExecutionEngine(study_name)
factory = exec_eng.factory
builder = factory.get_builder_from_process(repo=repo,mod_id=proc_name)
exec_eng.factory.set_builders_to_coupling_builder(builder)
exec_eng.configure()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe.SellarDoeScenario.SellarCoupling is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi


In [14]:
exec_eng.display_treeview_nodes()

INFO:SoS.EE:Nodes representation for Treeview doe
|_ doe
	|_ SellarDoeScenario
		|_ SellarCoupling
			|_ Sellar_2
			|_ Sellar_1
			|_ Sellar_Problem


'Nodes representation for Treeview doe\n|_ doe\n\t|_ SellarDoeScenario\n\t\t|_ SellarCoupling\n\t\t\t|_ Sellar_2\n\t\t\t|_ Sellar_1\n\t\t\t|_ Sellar_Problem'

In [15]:
exec_eng.dm.get_data('doe.sub_mda_class')['value']

'MDAJacobi'

Possible algo options:

In [16]:
from gemseo.algos.doe.doe_factory import DOEFactory
driver_lib = DOEFactory().create(my_doe_algo)
driver_lib.init_options_grammar(my_doe_algo)
algo_options_keys = list(driver_lib.opt_grammar.properties_dict.keys())
print(algo_options_keys)

['criterion', 'iterations', 'n_samples', 'eval_jac', 'n_processes', 'wait_time_between_samples', 'seed', 'max_time']


We can use or not eval mode

In [17]:
#-- set up disciplines in Scenario
disc_dict = {}
# Doe inputs
disc_dict[f'{ns}.SellarDoeScenario.n_samples'] = 100
# 'lhs', 'CustomDOE', 'fullfact', ...
disc_dict[f'{ns}.SellarDoeScenario.algo'] = my_doe_algo
disc_dict[f'{ns}.SellarDoeScenario.design_space'] = dspace

disc_dict[f'{ns}.SellarDoeScenario.formulation'] = my_formulation
disc_dict[f'{ns}.SellarDoeScenario.objective_name'] = 'obj'
#disc_dict[f'{ns}.SellarDoeScenario.ineq_constraints'] = [f'c_1', f'c_2']
disc_dict[f'{ns}.SellarDoeScenario.algo_options'] = {'levels': 'None'
                                            }
#disc_dict[f'{ns}.SellarDoeScenario.eval_mode'] = True

disc_dict

{'doe.SellarDoeScenario.n_samples': 100,
 'doe.SellarDoeScenario.algo': 'lhs',
 'doe.SellarDoeScenario.design_space':   variable       value     lower_bnd     upper_bnd  enable_variable  \
 0        x       [1.0]         [0.0]        [10.0]             True   
 1        z  [5.0, 2.0]  [-10.0, 0.0]  [10.0, 10.0]             True   
 
   activated_elem  
 0         [True]  
 1   [True, True]  ,
 'doe.SellarDoeScenario.formulation': 'DisciplinaryOpt',
 'doe.SellarDoeScenario.objective_name': 'obj',
 'doe.SellarDoeScenario.algo_options': {'levels': 'None'}}

Remark: in fact it is a <b> DoE optim </b> (with 'obj' variable taken as obj function) ! Here dspace variable was used.

In [18]:
pd.DataFrame.from_dict(disc_dict,orient='index')

,0
doe.SellarDoeScenario.n_samples,100
doe.SellarDoeScenario.algo,lhs
doe.SellarDoeScenario.design_space,variable value lower_bnd upper...
doe.SellarDoeScenario.formulation,DisciplinaryOpt
doe.SellarDoeScenario.objective_name,obj
doe.SellarDoeScenario.algo_options,{'levels': 'None'}


Here : use of load_study_from_input_dict() method

In [19]:
exec_eng.load_study_from_input_dict(disc_dict)

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe.SellarDoeScenario.SellarCoupling is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe.SellarDoeScenario.SellarCoupling is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi
ERROR:SoS.EE.DataManager:Variable: doe.SellarDoeScenario.SellarCoupling.x value is not set!
ERROR:SoS.EE.DataManager:Variable: doe.SellarDoeScenario.SellarCoupling.z value is not set!
ERROR:SoS.EE.DataManager:Variable: doe.SellarDoeScenario.SellarCoupling.Sellar_Problem.local_dv value is not set!


# 2 Display Disciplines I/O variables

Here "SellarCoupling" has been added in name:

In [20]:
sellar1 = exec_eng.dm.get_disciplines_with_name("doe." + "SellarDoeScenario.SellarCoupling.Sellar_1")[0]

In [21]:
type(sellar1)

sos_trades_core.sos_wrapping.test_discs.sellar.Sellar1

In [22]:
DESC_IN = sellar1.DESC_IN
DESC_OUT = sellar1.DESC_OUT
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [23]:
DESC_IN_df

,type,visibility,namespace
x,float,Shared,ns_OptimSellar
y_2,float,Shared,ns_OptimSellar
z,array,Shared,ns_OptimSellar


In [24]:
DESC_OUT_df

,type,visibility,namespace
y_1,float,Shared,ns_OptimSellar
y_1_bis,float,Shared,ns_OptimSellar


In [25]:
sellar2 = exec_eng.dm.get_disciplines_with_name("doe." + "SellarDoeScenario.SellarCoupling.Sellar_2")[0]

In [26]:
DESC_IN = sellar2.DESC_IN
DESC_OUT = sellar2.DESC_OUT
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [27]:
DESC_IN_df

,type,visibility,namespace,default
y_1,float,Shared,ns_OptimSellar,NaN
z,array,Shared,ns_OptimSellar,NaN
debug_mode_sellar,bool,Shared,ns_OptimSellar,False


In [28]:
DESC_OUT_df

,type,visibility,namespace
y_2,float,Shared,ns_OptimSellar
y_2_bis,float,Shared,ns_OptimSellar


In [29]:
sellarP = exec_eng.dm.get_disciplines_with_name("doe." + "SellarDoeScenario.SellarCoupling.Sellar_Problem")[0]

In [30]:
DESC_IN = sellarP.DESC_IN
DESC_OUT = sellarP.DESC_OUT
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [31]:
DESC_IN_df

,type,visibility,namespace
x,float,Shared,ns_OptimSellar
y_1,float,Shared,ns_OptimSellar
y_2,float,Shared,ns_OptimSellar
z,array,Shared,ns_OptimSellar
local_dv,float,NaN,NaN


In [32]:
DESC_OUT_df

,type,visibility,namespace
c_1,array,Shared,ns_OptimSellar
c_2,array,Shared,ns_OptimSellar
obj,array,Shared,ns_OptimSellar


Here we also have SellarCoupling discipline:

In [33]:
sellarCoupling = exec_eng.dm.get_disciplines_with_name("doe." + "SellarDoeScenario.SellarCoupling")[0]

In [34]:
type(sellarCoupling)

sos_trades_core.execution_engine.sos_coupling.SoSCoupling

In [35]:
DESC_IN = sellar2.DESC_IN
DESC_OUT = sellar2.DESC_OUT
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [36]:
DESC_IN_df

,type,visibility,namespace,default
y_1,float,Shared,ns_OptimSellar,NaN
z,array,Shared,ns_OptimSellar,NaN
debug_mode_sellar,bool,Shared,ns_OptimSellar,False


In [37]:
DESC_OUT_df 

,type,visibility,namespace
y_2,float,Shared,ns_OptimSellar
y_2_bis,float,Shared,ns_OptimSellar


# 3 Provide Discipline inputs

In [38]:
# Sellar inputs
local_dv = 10.
values_dict = {}
# array([1.])
values_dict[f'{ns}.{sc_name}.{c_name}.sub_mda_class'] = 'MDANewtonRaphson'
values_dict[f'{ns}.{sc_name}.{c_name}.x'] = 2.
values_dict[f'{ns}.{sc_name}.{c_name}.y_1'] = 2.
values_dict[f'{ns}.{sc_name}.{c_name}.y_2'] = 2.
values_dict[f'{ns}.{sc_name}.{c_name}.z'] = array([2., 2.])
values_dict[f'{ns}.{sc_name}.{c_name}.Sellar_Problem.local_dv'] = local_dv
values_dict
my_dict = exec_eng.get_anonimated_data_dict()
pd.DataFrame.from_dict(values_dict,orient='index')

,0
doe.SellarDoeScenario.SellarCoupling.sub_mda_class,MDANewtonRaphson
doe.SellarDoeScenario.SellarCoupling.x,2.0
doe.SellarDoeScenario.SellarCoupling.y_1,2.0
doe.SellarDoeScenario.SellarCoupling.y_2,2.0
doe.SellarDoeScenario.SellarCoupling.z,"[2.0, 2.0]"
doe.SellarDoeScenario.SellarCoupling.Sellar_Problem.local_dv,10.0


In [39]:
#exec_eng.dm.set_values_from_dict(values_dict)
exec_eng.load_study_from_input_dict(values_dict)
exec_eng.configure()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe.SellarDoeScenario.SellarCoupling is set to MDANewtonRaphson
INFO:SoS.EE:configuring ...
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi


# 4 Run Doe scenario

In [40]:
res = exec_eng.execute()
# res is without interest (return not the expected values but probably default values ) !

INFO:SoS.EE:PROCESS EXECUTION doe STARTS...
INFO:gemseo.core.doe_scenario: 
INFO:gemseo.core.doe_scenario:*** Start DOE Scenario execution ***
INFO:gemseo.core.doe_scenario:SellarDoeScenario
   Disciplines: SellarCoupling
   MDOFormulation: DisciplinaryOpt
   Algorithm: None
INFO:SoS.EE.SoSDOEScenario:{'iterations': 5, 'eval_jac': False, 'n_processes': 1, 'wait_time_between_samples': 0.0, 'seed': 1, 'max_time': 0}
INFO:gemseo.algos.doe.doe_lib:Optimization problem:
   Minimize: doe.SellarDoeScenario.SellarCoupling.obj(doe.SellarDoeScenario.SellarCoupling.x, doe.SellarDoeScenario.SellarCoupling.z)
   With respect to: doe.SellarDoeScenario.SellarCoupling.x, doe.SellarDoeScenario.SellarCoupling.z
INFO:gemseo.algos.driver_lib:DOE sampling:   0%|          | 0/100 [00:00<?, ?it]
INFO:SoS.EE.Coupling:doe.SellarDoeScenario.SellarCoupling MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	1.151857e+07
INFO:gemseo.mda.mda:	2	2.164783e+06
INFO:gemseo.mda.mda:	3	3.061206e+03
IN

my_options
{'iterations': 5, 'eval_jac': False, 'n_processes': 1, 'wait_time_between_samples': 0.0, 'seed': 1, 'max_time': 0}


INFO:gemseo.mda.mda:	3	2.595664e+03
INFO:gemseo.mda.mda:	4	4.023349e+00
INFO:gemseo.mda.mda:	5	7.730185e-03
INFO:gemseo.mda.mda:	6	1.197119e-05
INFO:gemseo.mda.mda:	7	2.364000e-06
INFO:gemseo.mda.mda:	8	3.567449e-07
INFO:SoS.EE.Coupling:doe.SellarDoeScenario.SellarCoupling MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	4.555212e+06
INFO:gemseo.mda.mda:	2	1.006180e+06
INFO:gemseo.mda.mda:	3	1.449587e+03
INFO:gemseo.mda.mda:	4	2.909055e+00
INFO:gemseo.mda.mda:	5	6.512315e-03
INFO:gemseo.mda.mda:	6	1.302487e-03
INFO:gemseo.mda.mda:	7	2.858776e-04
INFO:gemseo.mda.mda:	8	5.717646e-05
INFO:gemseo.mda.mda:	9	1.254932e-05
INFO:gemseo.mda.mda:	10	2.509909e-06
INFO:gemseo.mda.mda:	11	5.508400e-07
INFO:gemseo.algos.driver_lib:DOE sampling:   4%|▍         | 4/100 [00:00<00:00, 252.94 it/sec, obj=18.8]
INFO:SoS.EE.Coupling:doe.SellarDoeScenario.SellarCoupling MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	7.160444e+07
INFO:gemseo.mda.mda:	2	6.244855e+

INFO:gemseo.algos.driver_lib:DOE sampling:  19%|█▉        | 19/100 [00:02<00:01, 44.89 it/sec, obj=26.7]
INFO:SoS.EE.Coupling:doe.SellarDoeScenario.SellarCoupling MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	7.763027e+06
INFO:gemseo.mda.mda:	2	1.527764e+06
INFO:gemseo.mda.mda:	3	3.038920e+03
INFO:gemseo.mda.mda:	4	4.265442e+00
INFO:gemseo.mda.mda:	5	8.516605e-03
INFO:gemseo.mda.mda:	6	1.213402e-05
INFO:gemseo.mda.mda:	7	2.418256e-06
INFO:gemseo.mda.mda:	8	3.398082e-07
INFO:gemseo.algos.driver_lib:DOE sampling:  20%|██        | 20/100 [00:02<00:01, 42.78 it/sec, obj=29.1]
INFO:SoS.EE.Coupling:doe.SellarDoeScenario.SellarCoupling MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	4.373821e+07
INFO:gemseo.mda.mda:	2	4.997488e+06
INFO:gemseo.mda.mda:	3	8.719785e+03
INFO:gemseo.mda.mda:	4	6.231941e+00
INFO:gemseo.mda.mda:	5	1.125861e-02
INFO:gemseo.mda.mda:	6	7.734498e-06
INFO:gemseo.mda.mda:	7	1.484067e-06
INFO:gemseo.mda.mda:	8	9.871175e-08
IN

INFO:SoS.EE.Coupling:doe.SellarDoeScenario.SellarCoupling MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	6.563294e+06
INFO:gemseo.mda.mda:	2	1.409243e+06
INFO:gemseo.mda.mda:	3	1.686089e+03
INFO:gemseo.mda.mda:	4	3.328952e+00
INFO:gemseo.mda.mda:	5	6.191592e-03
INFO:gemseo.mda.mda:	6	1.203055e-03
INFO:gemseo.mda.mda:	7	2.215484e-04
INFO:gemseo.mda.mda:	8	4.304848e-05
INFO:gemseo.mda.mda:	9	7.927834e-06
INFO:gemseo.mda.mda:	10	1.540202e-06
INFO:gemseo.mda.mda:	11	2.829967e-07
INFO:gemseo.algos.driver_lib:DOE sampling:  36%|███▌      | 36/100 [00:04<00:02, 22.65 it/sec, obj=35.7]
INFO:SoS.EE.Coupling:doe.SellarDoeScenario.SellarCoupling MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	7.349218e+07
INFO:gemseo.mda.mda:	2	6.263139e+06
INFO:gemseo.mda.mda:	3	1.250652e+04
INFO:gemseo.mda.mda:	4	6.103041e+00
INFO:gemseo.mda.mda:	5	1.218995e-02
INFO:gemseo.mda.mda:	6	6.017232e-06
INFO:gemseo.mda.mda:	7	1.186458e-06
INFO:gemseo.mda.mda:	8	5.864668e-

INFO:gemseo.mda.mda:	1	5.130263e+06
INFO:gemseo.mda.mda:	2	1.146366e+06
INFO:gemseo.mda.mda:	3	1.823878e+03
INFO:gemseo.mda.mda:	4	3.333910e+00
INFO:gemseo.mda.mda:	5	6.615767e-03
INFO:gemseo.mda.mda:	6	1.187358e-03
INFO:gemseo.mda.mda:	7	2.337333e-04
INFO:gemseo.mda.mda:	8	4.194883e-05
INFO:gemseo.mda.mda:	9	8.257950e-06
INFO:gemseo.mda.mda:	10	1.482313e-06
INFO:gemseo.mda.mda:	11	2.912494e-07
INFO:gemseo.algos.driver_lib:DOE sampling:  52%|█████▏    | 52/100 [00:06<00:03, 15.52 it/sec, obj=32.4]
INFO:SoS.EE.Coupling:doe.SellarDoeScenario.SellarCoupling MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	4.874736e+06
INFO:gemseo.mda.mda:	2	1.091318e+06
INFO:gemseo.mda.mda:	3	2.156425e+03
INFO:gemseo.mda.mda:	4	3.665653e+00
INFO:gemseo.mda.mda:	5	7.317072e-03
INFO:gemseo.mda.mda:	6	1.264166e-05
INFO:gemseo.mda.mda:	7	2.526428e-06
INFO:gemseo.mda.mda:	8	4.290357e-07
INFO:gemseo.algos.driver_lib:DOE sampling:  53%|█████▎    | 53/100 [00:06<00:03, 15.25 it/sec, obj=79.7

INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	1.878409e+07
INFO:gemseo.mda.mda:	2	2.843755e+06
INFO:gemseo.mda.mda:	3	5.215703e+03
INFO:gemseo.mda.mda:	4	5.261586e+00
INFO:gemseo.mda.mda:	5	1.003269e-02
INFO:gemseo.mda.mda:	6	9.964394e-06
INFO:gemseo.mda.mda:	7	1.972972e-06
INFO:gemseo.mda.mda:	8	1.908075e-07
INFO:gemseo.algos.driver_lib:DOE sampling:  69%|██████▉   | 69/100 [00:08<00:02, 11.99 it/sec, obj=91.2]
INFO:SoS.EE.Coupling:doe.SellarDoeScenario.SellarCoupling MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	2.465368e+07
INFO:gemseo.mda.mda:	2	3.790684e+06
INFO:gemseo.mda.mda:	3	5.273030e+03
INFO:gemseo.mda.mda:	4	5.988615e+00
INFO:gemseo.mda.mda:	5	9.463118e-03
INFO:gemseo.mda.mda:	6	1.028356e-05
INFO:gemseo.mda.mda:	7	1.774382e-06
INFO:gemseo.mda.mda:	8	1.860517e-07
INFO:gemseo.algos.driver_lib:DOE sampling:  70%|███████   | 70/100 [00:08<00:02, 11.76 it/sec, obj=74.9]
INFO:SoS.EE.Coupling:doe.SellarDoeScenario.SellarCoupling MDA history
IN

INFO:gemseo.mda.mda:	8	2.279316e-07
INFO:gemseo.algos.driver_lib:DOE sampling:  84%|████████▍ | 84/100 [00:10<00:01,  9.76 it/sec, obj=51.2]
INFO:SoS.EE.Coupling:doe.SellarDoeScenario.SellarCoupling MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	9.884806e+06
INFO:gemseo.mda.mda:	2	1.826424e+06
INFO:gemseo.mda.mda:	3	3.457623e+03
INFO:gemseo.mda.mda:	4	4.513285e+00
INFO:gemseo.mda.mda:	5	8.843692e-03
INFO:gemseo.mda.mda:	6	1.155732e-05
INFO:gemseo.mda.mda:	7	2.308451e-06
INFO:gemseo.mda.mda:	8	2.953223e-07
INFO:gemseo.algos.driver_lib:DOE sampling:  85%|████████▌ | 85/100 [00:10<00:01,  9.65 it/sec, obj=103]
INFO:SoS.EE.Coupling:doe.SellarDoeScenario.SellarCoupling MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	2.221195e+07
INFO:gemseo.mda.mda:	2	3.193643e+06
INFO:gemseo.mda.mda:	3	5.763957e+03
INFO:gemseo.mda.mda:	4	5.462886e+00
INFO:gemseo.mda.mda:	5	1.027843e-02
INFO:gemseo.mda.mda:	6	9.536493e-06
INFO:gemseo.mda.mda:	7	1.874213e-06
INF

INFO:gemseo.core.doe_scenario:*** DOE Scenario run terminated ***
INFO:SoS.EE:PROCESS EXECUTION doe ENDS.


Remark : all options that are None are no kept and so appear with a warning (this is the case of default None options thar are not setted with not None value)

# 5 Display Doe  Scenario results

In [41]:
# retrieve discipline to check the result...
doe_disc = exec_eng.dm.get_disciplines_with_name("doe." + sc_name)[0]

In [42]:
type(doe_disc)

sos_trades_core.execution_engine.sos_doe_scenario.SoSDOEScenario

In [43]:
doe_disc.get_disc_full_name()

'doe.SellarDoeScenario'

In [44]:
opt_P = doe_disc.opt_problem
print(opt_P)

Optimization problem:
   Minimize: doe.SellarDoeScenario.SellarCoupling.obj(doe.SellarDoeScenario.SellarCoupling.x, doe.SellarDoeScenario.SellarCoupling.z)
   With respect to: doe.SellarDoeScenario.SellarCoupling.x, doe.SellarDoeScenario.SellarCoupling.z


In [45]:
doe_disc_input = doe_disc.get_sosdisc_inputs()
pd.DataFrame.from_dict(doe_disc_input,orient='index')

,0
n_samples,100
algo,lhs
design_space,variable value lower_bnd upper...
formulation,DisciplinaryOpt
maximize_objective,False
objective_name,obj
differentiation_method,finite_differences
algo_options,"{'iterations': 5, 'eval_jac': False, 'n_proces..."
parallel_options,"{'parallel': False, 'n_processes': 8, 'use_thr..."
eval_mode,False


In [46]:
doe_disc_output = doe_disc.get_sosdisc_outputs()

In [47]:
# doe_disc_output

# 5.1 Table of computed points

In [48]:
XY_pd= doe_disc_output['doe_ds_io']

In [49]:
d = {'design_parameters': ['Eval mode as not io table of outputs'],'functions': ['Eval mode as not io table of outputs']}
XY_pd = pd.DataFrame(data=d)
XY_pd 

,design_parameters,functions
0,Eval mode as not io table of outputs,Eval mode as not io table of outputs


In [50]:
X_pd = XY_pd['design_parameters']
X_pd

0    Eval mode as not io table of outputs
Name: design_parameters, dtype: object

In [51]:
X_pd.to_csv('X_pd.csv',index=False)

In [52]:
Y_pd = XY_pd['functions']
Y_pd

0    Eval mode as not io table of outputs
Name: functions, dtype: object

# 5.2 Optimum

In [53]:
my_optim_result = doe_disc_output['optim_result']
my_optim_result

{'x_opt': array([ 1.00983468, -0.65057567,  0.76923226]),
 'f_opt': 13.915272626018334}

In [54]:
my_optim_result['x_opt']

array([ 1.00983468, -0.65057567,  0.76923226])

In [55]:
my_optim_result['f_opt']

13.915272626018334

# 5.3 From gemseo 

In [56]:
print(opt_P.objective)
print(opt_P.constraints)
print(opt_P.differentiation_method)
print(opt_P.fd_step)

doe.SellarDoeScenario.SellarCoupling.obj(doe.SellarDoeScenario.SellarCoupling.x, doe.SellarDoeScenario.SellarCoupling.z)
[]
finite_differences
1e-06


Remark: eval mode has no opt_P.export_to_dataset("dataset_name")

In [57]:
dataset = opt_P.export_to_dataset("dataset_name")

In [58]:
XY_pd=dataset.export_to_dataframe()
XY_pd

design_parameters  \
   doe.SellarDoeScenario.SellarCoupling.x   
                                        0   
0                                9.379764   
1                                8.886448   
2                                3.713714   
3                                0.041702   
4                                6.954955   
..                                    ...   
95                               6.661678   
96                               2.358656   
97                               3.011475   
98                               4.804455   
99                               4.001988   

                                                     \
   doe.SellarDoeScenario.SellarCoupling.z             
                                        0         1   
0                                1.515949  5.631736   
1                               -1.196271  6.523436   
2                               -1.994758  4.822571   
3                                1.749067  3.617234   
4                               -9.316161  9.918161   
..                                    ...       ...   
95                               5.719822  7.645988   
96                              -6.389327  5.297002   
97                               8.776000  9.837792   
98                              -2.413481  1.131552   
99                               2.954436  9.746571   

                                  functions  
   doe.SellarDoeScenario.SellarCoupling.obj  
                                          0  
0                                118.714541  
1                                110.493389  
2                                 39.894420  
3                                 18.808762  
4                                169.817929  
..                                      ...  
95                               105.062729  
96                                68.185137  
97                               113.215059  
98                                45.682186  
99                                54.826284  

[100 rows x 4 columns]

In [59]:
X_pd = XY_pd['design_parameters']
X_pd

doe.SellarDoeScenario.SellarCoupling.x  \
                                        0   
0                                9.379764   
1                                8.886448   
2                                3.713714   
3                                0.041702   
4                                6.954955   
..                                    ...   
95                               6.661678   
96                               2.358656   
97                               3.011475   
98                               4.804455   
99                               4.001988   

   doe.SellarDoeScenario.SellarCoupling.z            
                                        0         1  
0                                1.515949  5.631736  
1                               -1.196271  6.523436  
2                               -1.994758  4.822571  
3                                1.749067  3.617234  
4                               -9.316161  9.918161  
..                                    ...       ...  
95                               5.719822  7.645988  
96                              -6.389327  5.297002  
97                               8.776000  9.837792  
98                              -2.413481  1.131552  
99                               2.954436  9.746571  

[100 rows x 3 columns]

In [60]:
Y_pd = XY_pd['functions']
Y_pd

,doe.SellarDoeScenario.SellarCoupling.obj
,0
0,118.714541
1,110.493389
2,39.894420
3,18.808762
4,169.817929
...,...
95,105.062729
96,68.185137
97,113.215059


In [61]:
import numpy as np
min(np.array(Y_pd['doe.SellarDoeScenario.SellarCoupling.obj']['0']))

13.915272626018334

In [62]:
doe_disc.optimization_result.f_opt

13.915272626018334

In [63]:
doe_disc.optimization_result.x_opt

array([ 1.00983468, -0.65057567,  0.76923226])

In [64]:
doe_disc.print_execution_metrics()

INFO:gemseo.core.scenario:* Scenario Executions statistics *
INFO:gemseo.core.scenario:* Discipline: SellarCoupling
INFO:gemseo.core.scenario:Executions number: 100
INFO:gemseo.core.scenario:Execution time:  11.614314899999993 s
INFO:gemseo.core.scenario:Linearizations number: 0
INFO:gemseo.core.scenario:Total number of executions calls 100
INFO:gemseo.core.scenario:Total number of linearizations 0


In [65]:
doe_disc.get_optimum().__dict__

{'x_0': array([9.37976388, 1.51594904, 5.63173624]),
 'optimizer_name': 'lhs',
 'x_opt': array([ 1.00983468, -0.65057567,  0.76923226]),
 'message': None,
 'f_opt': 13.915272626018334,
 'constraints_values': {},
 'constraints_grad': {},
 'status': None,
 'n_obj_call': 100,
 'n_grad_call': None,
 'n_constr_call': None,
 'is_feasible': True}